In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

from functools import partial

from optimization import jg_opt
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.population import WorldpopPopulation
from pisa.population_served_by_isopolygons import get_population_served_by_isopolygons
from pisa.visualisation import (
    plot_facilities,
    plot_isochrones,
    plot_population,
    plot_population_heatmap,
)


In [ ]:
import os

from dotenv import load_dotenv

# load environment variables from an `.env` file in the local root directory
load_dotenv()

CBC_SOLVER_PATH = os.getenv(
    "CBC_SOLVER_PATH"
)  # path to the cbc executable (e.g. /opt/homebrew/bin/cbc)


### Define Administrative Area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm).

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [ ]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

### Facilities

#### Get existing facilities (in our case, hospitals) from OSM

In [ ]:
hospitals_df = Facilities(admin_area_boundaries=baucau).get_existing_facilities()

In [ ]:
plot_facilities(hospitals_df, baucau)

#### Estimate potential locations for new facilities


In [ ]:
potential_hospitals_df = Facilities(
    admin_area_boundaries=baucau
).estimate_potential_facilities(spacing=0.05)

### Get population 
In this example, from WorldPop

In [ ]:
population_gdf = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_gdf.head()

In [ ]:
plot_population_heatmap(population_gdf, baucau)

In [ ]:
plot_population(population_gdf, baucau, random_sample_n=1000)

### Calculate isopolygons

Here we make some choices:
- distance type
- distance values
- mode of transport 

Valid values for constants are in the script pisa.constants

In [ ]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [2000, 5000, 10000]

MODE_OF_TRANSPORT = "driving"

#### Using OSM

For this, we need to get the road network from osmnx

In [ ]:
from pisa.osm_road_network import OsmRoadNetwork

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau,
    mode_of_transport=MODE_OF_TRANSPORT,
    distance_type=DISTANCE_TYPE,
).get_osm_road_network()

#### Calculate isopolygons for existing facilities

In [ ]:
from pisa.isopolygons import OsmIsopolygonCalculator

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    isopolygons_existing_facilities = OsmIsopolygonCalculator(
        facilities_df=hospitals_df,
        distance_type=DISTANCE_TYPE,
        distance_values=DISTANCE_VALUES,
        road_network=road_network,
    ).calculate_isopolygons()

In [ ]:
isopolygons_existing_facilities.head()

In [ ]:
plot_isochrones(isopolygons_existing_facilities, baucau)

#### Calculate isopolygons for potential facilities

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    isopolygons_potential_facilities = OsmIsopolygonCalculator(
    facilities_df=potential_hospitals_df,
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    road_network=road_network,
    ).calculate_isopolygons()

## Prepare optimization data

Preparing the variables that go into the cell below

In [ ]:
population_served_current = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_existing_facilities
)

current = {DISTANCE_TYPE: population_served_current}

In [ ]:
population_served_potential = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_potential_facilities
)

potential = {DISTANCE_TYPE: population_served_potential}

In [ ]:
population_count = population_gdf.population.values

In [ ]:
BUDGET = [
    5,
    20,
    50,
]  # budget for the optimization in terms of how many locations can be built

cbc_optimize = partial(jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH)

values, solutions = jg_opt.Solve(
    household=population_count,
    current=current,
    potential=potential,
    accessibility=DISTANCE_TYPE,
    budgets=BUDGET,
    optimize=cbc_optimize,
    type="ID",
)

In [ ]:
values

In [ ]:
solutions